In [1]:
import csv
import glob
import os
import tensorflow as tf
gpus = tf.config.list_physical_devices("GPU")
def csv_gen(image_folder, image_format, label_folder, label_format, output_csv):
    rows = []
    image_paths = sorted(glob.glob(os.path.join(image_folder, f"*.{image_format}")))
    label_paths = sorted(glob.glob(os.path.join(label_folder, f"*.{label_format}")))
    print(f"Number of images: {len(image_paths)}, Number of labels: {len(label_paths)}")
    
    for i, j in zip(image_paths, label_paths):
        if os.path.splitext(os.path.basename(i))[0] == os.path.splitext(os.path.basename(j))[0]:
            rows.append([i, j])
        else:
            print(f"Image and label names match: {os.path.basename(i)} and {os.path.basename(j)}")
            rows.append([i, j])

    with open(output_csv, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerows(rows)
    
    print("CSV file created")

# Parameters
image_folder = "scene/scene1"
image_format = "tif"
label_folder = "boundary/boundary1"
label_format = "tif"
output_csv = "output.csv"

# Generate CSV
csv_gen(image_folder, image_format, label_folder, label_format, output_csv)


Number of images: 49, Number of labels: 49
Image and label names match: scene1.0.tif and boundary1.0.tif
Image and label names match: scene1.1.tif and boundary1.1.tif
Image and label names match: scene1.105.tif and boundary1.105.tif
Image and label names match: scene1.106.tif and boundary1.106.tif
Image and label names match: scene1.107.tif and boundary1.107.tif
Image and label names match: scene1.108.tif and boundary1.108.tif
Image and label names match: scene1.109.tif and boundary1.109.tif
Image and label names match: scene1.110.tif and boundary1.110.tif
Image and label names match: scene1.111.tif and boundary1.111.tif
Image and label names match: scene1.126.tif and boundary1.126.tif
Image and label names match: scene1.127.tif and boundary1.127.tif
Image and label names match: scene1.128.tif and boundary1.128.tif
Image and label names match: scene1.129.tif and boundary1.129.tif
Image and label names match: scene1.130.tif and boundary1.130.tif
Image and label names match: scene1.131.t

In [2]:
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D,Activation,concatenate

from keras.layers import Dense, Flatten,BatchNormalization,UpSampling2D,Add
from keras.models import Model
def buildmodel_unet(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    c1 = Conv2D(64, (1,1), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (1,1), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(128, (1,1), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (1,1), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    
    c3 = Conv2D(256, (1,1), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (1,1), activation='relu', padding='same')(c3)
    c3 = Conv2D(256, (1,1), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    
    c4 = Conv2D(512, (1,1), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (1,1), activation='relu', padding='same')(c4)
    c4 = Conv2D(512, (1,1), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)
    
    # Bottleneck
    
    c5 = Conv2D(512, (1,1), activation='relu', padding='same')(p4)
    c5 = Conv2D(512, (1,1), activation='relu', padding='same')(c5)
    c5 = Conv2D(512, (1,1), activation='relu', padding='same')(c5)
    
    #sepeartion------
    
    p5 = MaxPooling2D((2, 2))(c5)
    
    # center_blocks
    
    cb1 = Conv2D(512, (1,1), padding='same')(p5)
    bn1 = BatchNormalization()(cb1)
    act1 = Activation('relu')(bn1)
    
    cb2 = Conv2D(512, (1,1), padding='same')(act1)
    bn2 = BatchNormalization()(cb2)
    act2 = Activation('relu')(bn2)
    
    #Decoder_stage
    
    up1 = UpSampling2D((2, 2))(act2)
    
    u1 = concatenate([up1, c5])
    
    c6 = Conv2D(256,(1,1), padding='same')(u1)
    bn3 = BatchNormalization()(c6)
    act3 = Activation('relu')(bn3)
    
    c7 = Conv2D(256, (1,1) , padding='same')(act3)
    bn4 = BatchNormalization()(c7)
    act4 = Activation('relu')(bn4)
    
    up2 = UpSampling2D((2, 2))(act4)
    
    u2 = concatenate([up2, c4])
    
    c8 = Conv2D(128, (1,1) , padding='same')(u2)
    bn5 = BatchNormalization()(c8)
    act5 = Activation('relu')(bn5)
    
    c9 = Conv2D(128, (1,1) , padding='same')(act5)
    bn6 = BatchNormalization()(c9)
    act6 = Activation('relu')(bn6)
    
    up3 = UpSampling2D((2, 2))(act6)
    
    u3 = concatenate([up3, c3])
    
    c10 = Conv2D(64, (1,1), padding='same')(u3)
    bn7 = BatchNormalization()(c10)
    act7 = Activation('relu')(bn7)
    
    c11 = Conv2D(64, (1,1), padding='same')(act7)
    bn8 = BatchNormalization()(c11)
    act8 = Activation('relu')(bn8)
    
    up4 = UpSampling2D((2, 2))(act8)
    
    u4 = concatenate([up4, c2])
    
    c12 = Conv2D(32, (1,1) , padding='same')(u4)
    bn9 = BatchNormalization()(c12)
    act9 = Activation('relu')(bn9)
    
    c13 = Conv2D(32, (1,1), padding='same')(act9)
    bn10 = BatchNormalization()(c13)
    act10 = Activation('relu')(bn10)
    
    up5 = UpSampling2D((2, 2))(act10)
    
    c14 = Conv2D(16, (1,1), padding='same')(up5)
    bn11 = BatchNormalization()(c14)
    act11 = Activation('relu')(bn11)
    
    c15 = Conv2D(16, (1,1) , padding='same')(act11)
    bn12 = BatchNormalization()(c15)
    act12 = Activation('relu')(bn12)
    
    c16 = Conv2D(num_classes,(1,1))(act12)
    outputs=Activation('sigmoid')(c16)
    
    model = Model(inputs, outputs)
    return model

In [3]:
def fpn_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)
    #--------block1-----
    
    c1 = Conv2D(64, (1,1), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (1,1), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    #--------block2---------
    
    c2 = Conv2D(128, (1,1), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (1,1), activation='relu', padding='same')(c2)
    
    #------1st-------------
    
    p2 = MaxPooling2D((2, 2))(c2)
    
    #--------block3-----------
    
    c3 = Conv2D(256, (1,1), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (1,1), activation='relu', padding='same')(c3)
    c3 = Conv2D(256, (1,1), activation='relu', padding='same')(c3)
    
    #------2nd----------------
    
    p3 = MaxPooling2D((2, 2))(c3)
    
    #----------block4-------
    
    c4 = Conv2D(512, (1,1), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (1,1), activation='relu', padding='same')(c4)
    c4 = Conv2D(512, (1,1), activation='relu', padding='same')(c4)
    
    #-----------3rd-----------
    
    p4 = MaxPooling2D((2, 2))(c4)
    
    #-------------block5------------
    
    c5 = Conv2D(512, (1,1), activation='relu', padding='same')(p4)
    c5 = Conv2D(512, (1,1), activation='relu', padding='same')(c5)
    c5 = Conv2D(512, (1,1), activation='relu', padding='same')(c5)

    #-----------4th------------
    
    p5 = MaxPooling2D((2, 2))(c5)
    
    p5_pre_conv=Conv2D(256,(1,1),activation='relu', padding='same')(p5)
    p5_upsampling=UpSampling2D((2, 2))(p5_pre_conv)
    
    fpn_stage_p5_conv=Conv2D(256,(1,1),activation='relu', padding='same')(c5)
    #-----------4th------------END
    p5_add= Add()([fpn_stage_p5_conv, p5_upsampling])
    
    
    
    #-------------------5---1th------------
    
    segm_5_a_conv = Conv2D(128, (1,1), padding='same')(p5_add)
    seg_5_a_bn1 = BatchNormalization()(segm_5_a_conv)
    seg_5_a_act1 = Activation('relu')(seg_5_a_bn1)
    
    segm_5_b_conv = Conv2D(128, (1,1), padding='same')(seg_5_a_act1)
    seg_5_b_bn1 = BatchNormalization()(segm_5_b_conv)
    seg_5_b_act1 = Activation('relu')(seg_5_b_bn1)
    
    #----------------5th---->1th-->END----
    
    upsampling_stage5= UpSampling2D((8, 8))(seg_5_b_act1)
    
    #-------------------5th---2nd------------
    
    p4_upsampling = UpSampling2D((2, 2))(p5_add)
    
    fpn_stage_p4_conv = Conv2D(256, (1,1), activation='relu', padding='same')(c4)
     #---------------3rd------------END
    p4_add = Add()([p4_upsampling, fpn_stage_p4_conv])
    
    #-------------------5th-2nd->1st---------------
    
    seg_stage4a_conv = Conv2D(128, (1,1), padding='same')(p4_add)
    seg_stage4a_bn = BatchNormalization()(seg_stage4a_conv)
    seg_stage4a_relu= Activation('relu')(seg_stage4a_bn)
    
    seg_stage4b_conv = Conv2D(128, (1,1), padding='same')(seg_stage4a_relu)
    seg_stage4b_bn = BatchNormalization()(seg_stage4b_conv)
    seg_stage4b_relu= Activation('relu')(seg_stage4b_bn)
    
    #---------------------5th--2nd->1st->end---------------
    
    upsampling_stage4 = UpSampling2D((4,4))(seg_stage4b_relu)
   
    #----------5th---2nd->2nd-------------------------
    
    fpn_stage_p3_upsampling = UpSampling2D((2, 2))(p4_add)
    fpn_stage_p3_conv = Conv2D(256, (1,1), activation='relu', padding='same')(c3)
    #----------------------2nd-----------end
    p3_add =  Add()([fpn_stage_p3_upsampling, fpn_stage_p3_conv])
    
    
    #----------------5th---2nd->2nd--->1st-----------------
    
    segm_stage3a_conv = Conv2D(128, (1,1), padding='same')(p3_add)
    segm_stage3a_bn = BatchNormalization()(segm_stage3a_conv)
    segm_stage3a_relu = Activation('relu')(segm_stage3a_bn)
    
    segm_stage3b_conv = Conv2D(128, (1,1), padding='same')(segm_stage3a_relu)
    segm_stage3b_bn = BatchNormalization()(segm_stage3b_conv)
    segm_stage3b_relu = Activation('relu')(segm_stage3b_bn)
    
    #----------------5th---2nd->2nd--->1st-----------END
    
    upsampling_stage3 = UpSampling2D((2, 2))(segm_stage3b_relu)
    
    #----------------5th---2nd->2nd--->2nd-----------------
    
    fpn_stage_p2_upsampling = UpSampling2D((2, 2))(p3_add)
    fpn_stage_p2_conv = Conv2D(256, (1,1), activation='relu', padding='same')(c2)
    #------------------1st---------------END
    p2_add = Add()([fpn_stage_p2_upsampling, fpn_stage_p2_conv])
    
    segm_stage2a_conv = Conv2D(128, (1,1), padding='same')(p2_add)
    segm_stage2a_bn = BatchNormalization()(segm_stage2a_conv)
    segm_stage2a_relu = Activation('relu')(segm_stage2a_bn)
    
    segm_stage2b_conv = Conv2D(128, (1,1), padding='same')(segm_stage2a_relu)
    segm_stage2b_bn = BatchNormalization()(segm_stage2b_conv)
    #----------------5th---2nd->2nd--->2nd-----------------END
    segm_stage2b_relu = Activation('relu')(segm_stage2b_bn)
    
    #-------------------Final-------------
    aggregation_concat = concatenate([upsampling_stage5,upsampling_stage4,upsampling_stage3,segm_stage2b_relu])
    
    final_stage_conv = Conv2D(128, (1,1), padding='same')(aggregation_concat)
    final_stage_bn = BatchNormalization()(final_stage_conv)
    final_stage_relu = Activation('relu')(final_stage_bn)
    final_upsampling = UpSampling2D((2, 2))(final_stage_relu)
    head_conv = Conv2D( num_classes, (1,1), padding='same')(final_upsampling)
    outputs=Activation('sigmoid')(head_conv)
    
    model = Model(inputs,outputs)
    
    return model

In [4]:
import numpy as np
from keras.utils import Sequence
import gdal
import csv
import tensorflow as tf

class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, image_paths, label_paths, batch_size=32, n_classes=2, n_channels=3, patch_size=(128, 128),
                 shuffle=True, rs=255, rs_label=1):
        'Initialization'
        self.batch_size = batch_size
        self.label_paths = label_paths
        self.image_paths = image_paths
        self.n_classes = n_classes
        self.n_channels = n_channels
        self.patch_size = patch_size
        self.shuffle = shuffle
        self.rescale_value = rs
        self.rs_label = rs_label
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.ceil(len(self.image_paths) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        list_image_temp = [self.image_paths[k] for k in indexes]
        list_label_temp = [self.label_paths[k] for k in indexes]
        X, y = self.__data_generation(list_image_temp, list_label_temp)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.image_paths))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, image_paths, label_paths):
        'Generates data containing batch_size samples'
        X = np.empty((self.batch_size, *self.patch_size, self.n_channels))
        y = np.empty((self.batch_size, *self.patch_size, 1), dtype=int)
        for i, (image, label) in enumerate(zip(image_paths, label_paths)):
            _image = gdal.Open(image)
            _label = gdal.Open(label)
            if _image is None or _label is None:
                print(f"Failed to open image or label file: {image}, {label}")
                continue
            _image = np.array(_image.ReadAsArray()) / self.rescale_value
            _image = _image.transpose(1, 2, 0)
            _label = np.array(_label.ReadAsArray()) / self.rs_label
            _label = np.expand_dims(_label, axis=-1)
            X[i,] = _image
            y[i,] = _label
        return X, y


In [5]:
# import numpy as np
# from keras.callbacks import ModelCheckpoint, CSVLogger
# import datetime
# import csv
# from sklearn.metrics import confusion_matrix, accuracy_score , f1_score
# def read_csv(csv_file):
#     image_paths = []
#     label_paths = []
#     with open(csv_file, 'r') as file:
#         reader = csv.reader(file)
#         for row in reader:
#             image_paths.append(row[0])
#             label_paths.append(row[1])
#     return image_paths, label_paths

# # Example usage:
# csv_file = "output.csv"
# image_paths, label_paths = read_csv(csv_file)

# # Train, Validation and Test Split
# train_val_split_index = int(len(image_paths) * 0.8)
# train_image_paths = image_paths[:train_val_split_index]
# train_label_paths = label_paths[:train_val_split_index]
# valid_test_image_paths = image_paths[train_val_split_index:]
# valid_test_label_paths = label_paths[train_val_split_index:]

# valid_split_index = int(len(valid_test_image_paths) * 0.5)
# valid_image_paths = valid_test_image_paths[:valid_split_index]
# valid_label_paths = valid_test_label_paths[:valid_split_index]
# test_image_paths = valid_test_image_paths[valid_split_index:]
# test_label_paths = valid_test_label_paths[valid_split_index:]

# def train(model_name, train_image_paths, train_label_paths, valid_image_paths, valid_label_paths, 
#           input_shape, batch_size, num_classes, epochs, rs, rs_label, weights=None):
#     rescale_value = 2 ** rs - 1
    
#     # Building the model
#     model = buildmodel_unet(input_shape, num_classes)
    
#     if weights:
#         model.load_weights(weights)
    
#     if num_classes > 1:
#         loss_fun = "categorical_crossentropy"
#     elif num_classes == 1:
#         loss_fun = "binary_crossentropy"
#     else:
#         raise ValueError("Number of classes not specified correctly")
    
#     model.compile(optimizer="adam", loss=loss_fun, metrics=["acc"])
    
#     # Custom data generator
#     train_gen = DataGenerator(
#         train_image_paths, 
#         train_label_paths,
#         batch_size=batch_size,
#         patch_size=(256, 256),
#         n_channels=3,
#         n_classes=num_classes,
#         rs=rs,
#         rs_label=rs_label,
#         shuffle=True
#     )
    
#     valid_gen = DataGenerator(
#         valid_image_paths, 
#         valid_label_paths,
#         batch_size=batch_size,
#         patch_size=(256, 256),
#         n_channels=3,
#         n_classes=num_classes,
#         rs=rs,
#         rs_label=rs_label,
#         shuffle=False
#     )
    
#     train_steps = len(train_image_paths) // batch_size
#     valid_steps = len(valid_image_paths) // batch_size
    
#     model_checkpoint = ModelCheckpoint(filepath='best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
#     csv_logger = CSVLogger('training_log.csv')
    
#     model.fit(
#         train_gen,
#         steps_per_epoch=train_steps,
#         validation_data=valid_gen,
#         validation_steps=valid_steps,
#         epochs=epochs,
#         callbacks=[model_checkpoint]
#     )
    
    
#     print("Model successfully trained")

#     return model

# def evaluate_model(model, test_image_paths, test_label_paths, batch_size, rs, rs_label):
#     test_gen = DataGenerator(
#         test_image_paths, 
#         test_label_paths,
#         batch_size=batch_size,
#         patch_size=(256, 256),
#         n_channels=3,
#         n_classes=1,
#         rs=rs,
#         rs_label=rs_label,
#         shuffle=False
#     )

#     # Get predictions from the model
#     predictions = model.predict(test_gen, verbose=1)

#     # Flatten predictions and true labels
#     pred_masks = (predictions > 0.5).astype(np.uint8).flatten()
#     true_masks = np.concatenate([test_gen[i][1].flatten() for i in range(len(test_gen))]).astype(np.uint8)

#     # Calculate confusion matrix
#     cm = confusion_matrix(true_masks, pred_masks)

#     # Check if confusion matrix is binary
#     if cm.shape == (2, 2):
#         tn, fp, fn, tp = cm.ravel()
#     else:
#         raise ValueError("Confusion matrix does not have the shape (2, 2). This is expected for binary classification.")

#     # Calculate accuracy
#     accuracy = accuracy_score(true_masks, pred_masks)

#     # Calculate precision, recall, and F1-score
#     precision = precision_score(true_masks, pred_masks)
#     recall = recall_score(true_masks, pred_masks)
#     f1 = f1_score(true_masks, pred_masks)

#     # Calculate mean intersection over union (mIoU)
#     intersection = np.logical_and(true_masks, pred_masks)
#     union = np.logical_or(true_masks, pred_masks)
#     iou = np.sum(intersection) / np.sum(union)

#     print("Evaluation Metrics:")
#     print(f"Accuracy: {accuracy}")
#     print(f"Precision: {precision}")
#     print(f"Recall: {recall}")
#     print(f"F1-score: {f1}")
#     print(f"Mean Intersection over Union (mIoU): {iou}")

# # Parameters
# model_name = "unet"
# input_shape = (256, 256, 3)
# batch_size = 64
# num_classes = 1
# epochs = 10
# rs = 255
# rs_label = 1
# weights = None

# # Train the model
# model = train(model_name, train_image_paths, train_label_paths, valid_image_paths, valid_label_paths, 
#               input_shape, batch_size, num_classes, epochs, rs, rs_label, weights)

# # Evaluate the model
# evaluate_model(model, test_image_paths, test_label_paths, batch_size, rs, rs_label)


In [6]:
import numpy as np
import csv
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf

def read_csv(csv_file):
    image_paths = []
    label_paths = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            image_paths.append(row[0])
            label_paths.append(row[1])
    return image_paths, label_paths

# Example usage:
csv_file = "output.csv"
image_paths, label_paths = read_csv(csv_file)

# Train, Validation, and Test Split
train_split_index = int(len(image_paths) * 0.7)
valid_split_index = int(len(image_paths) * 0.85)

train_image_paths = image_paths[:train_split_index]
train_label_paths = label_paths[:train_split_index]
valid_image_paths = image_paths[train_split_index:valid_split_index]
valid_label_paths = label_paths[train_split_index:valid_split_index]
test_image_paths = image_paths[valid_split_index:]
test_label_paths = label_paths[valid_split_index:]

def train(model_name, train_image_paths, train_label_paths, valid_image_paths, valid_label_paths, 
          input_shape, batch_size, num_classes, epochs, rs, rs_label, weights=None):
    rescale_value = 2 ** rs - 1
    
    # Building the model
    model = fpn_model(input_shape, num_classes)
    
    if weights:
        model.load_weights(weights)
    
    if num_classes > 1:
        loss_fun = "categorical_crossentropy"
    elif num_classes == 1:
        loss_fun = "binary_crossentropy"
    else:
        raise ValueError("Number of classes not specified correctly")
    
    model.compile(optimizer="adam", loss=loss_fun, metrics=["accuracy"])
    
    # Custom data generator
    train_gen = DataGenerator(
        train_image_paths, 
        train_label_paths,
        batch_size=batch_size,
        patch_size=(256, 256),
        n_channels=3,
        n_classes=num_classes,
        rs=rs,
        rs_label=rs_label,
        shuffle=True
    )
    
    valid_gen = DataGenerator(
        valid_image_paths, 
        valid_label_paths,
        batch_size=batch_size,
        patch_size=(256, 256),
        n_channels=3,
        n_classes=num_classes,
        rs=rs,
        rs_label=rs_label,
        shuffle=False
    )
    
    train_steps = len(train_image_paths) // batch_size
    valid_steps = len(valid_image_paths) // batch_size
    
    # Training the model
    model.fit(
        train_gen,
        steps_per_epoch=train_steps,
        validation_data=valid_gen,
        validation_steps=valid_steps,
        epochs=epochs
    )
    
    print("Model successfully trained")

    return model

def evaluate_model(model, test_image_paths, test_label_paths, batch_size, rs, rs_label):
    test_gen = DataGenerator(
        test_image_paths, 
        test_label_paths,
        batch_size=batch_size,
        patch_size=(256, 256),
        n_channels=3,
        n_classes=1,
        rs=rs,
        rs_label=rs_label,
        shuffle=False
    )

    # Get predictions from the model
    predictions = model.predict(test_gen, verbose=1)

    # Flatten predictions and true labels
    pred_masks = (predictions > 0.5).astype(np.uint8).flatten()
    true_masks = np.concatenate([test_gen[i][1].flatten() for i in range(len(test_gen))]).astype(np.uint8)

    pred_masks = np.clip(pred_masks, 0, 1)
    true_masks = np.clip(true_masks, 0, 1)

    # Calculate accuracy
    accuracy = accuracy_score(true_masks, pred_masks)

    # Calculate F1-score
    f1 = f1_score(true_masks, pred_masks, average='binary')

    # Calculate mean intersection over union (mIoU)
    intersection = np.logical_and(true_masks, pred_masks)
    union = np.logical_or(true_masks, pred_masks)
    iou = np.sum(intersection) / np.sum(union)

    print("Evaluation Metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"F1-score: {f1}")
    print(f"Mean Intersection over Union (mIoU): {iou}")

# Parameters
model_name = "unet"
input_shape = (256, 256, 3)
batch_size = 4
num_classes = 1
epochs = 10
rs = 255
rs_label = 1
weights = None

# Train the model
model = train(model_name, train_image_paths, train_label_paths, valid_image_paths, valid_label_paths, 
              input_shape, batch_size, num_classes, epochs, rs, rs_label, weights)

# Evaluate the model
evaluate_model(model, test_image_paths, test_label_paths, batch_size, rs, rs_label)


Epoch 1/50
8/8 [==============================] - 17s 2s/step - loss: -2741.5427 - accuracy: 0.1583 - val_loss: 259.6722 - val_accuracy: 0.2026
Epoch 2/50
8/8 [==============================] - 16s 2s/step - loss: -11289.2461 - accuracy: 0.0547 - val_loss: 1885.5404 - val_accuracy: 0.2026
Epoch 3/50
8/8 [==============================] - 18s 2s/step - loss: -16765.9863 - accuracy: 0.0801 - val_loss: -651.2761 - val_accuracy: 0.0073
Epoch 4/50
8/8 [==============================] - 17s 2s/step - loss: -20796.9395 - accuracy: 0.0750 - val_loss: -2007.1426 - val_accuracy: 0.0145
Epoch 5/50
8/8 [==============================] - 17s 2s/step - loss: -25039.8496 - accuracy: 0.0726 - val_loss: -6957.1650 - val_accuracy: 8.3923e-05
Epoch 6/50
8/8 [==============================] - 16s 2s/step - loss: -29975.5137 - accuracy: 0.0677 - val_loss: -8841.5430 - val_accuracy: 0.0000e+00
Epoch 7/50
8/8 [==============================] - 17s 2s/step - loss: -37241.8945 - accuracy: 0.0031 - val_loss: -9

In [8]:
import numpy as np
import csv
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf

def read_csv(csv_file):
    image_paths = []
    label_paths = []
    with open(csv_file, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            image_paths.append(row[0])
            label_paths.append(row[1])
    return image_paths, label_paths

# Example usage:
csv_file = "output.csv"
image_paths, label_paths = read_csv(csv_file)

# Train, Validation, and Test Split
train_split_index = int(len(image_paths) * 0.7)
valid_split_index = int(len(image_paths) * 0.85)

train_image_paths = image_paths[:train_split_index]
train_label_paths = label_paths[:train_split_index]
valid_image_paths = image_paths[train_split_index:valid_split_index]
valid_label_paths = label_paths[train_split_index:valid_split_index]
test_image_paths = image_paths[valid_split_index:]
test_label_paths = label_paths[valid_split_index:]

def train(model_name, train_image_paths, train_label_paths, valid_image_paths, valid_label_paths, 
          input_shape, batch_size, num_classes, epochs, rs, rs_label, weights=None):
    rescale_value = 2 ** rs - 1
    
    # Building the model
    model = fpn_model(input_shape, num_classes)
    
    if weights:
        model.load_weights(weights)
    
    if num_classes > 1:
        loss_fun = "categorical_crossentropy"
    elif num_classes == 1:
        loss_fun = "binary_crossentropy"
    else:
        raise ValueError("Number of classes not specified correctly")
    
    model.compile(optimizer="adam", loss=loss_fun, metrics=["accuracy"])
    
    # Custom data generator
    train_gen = DataGenerator(
        train_image_paths, 
        train_label_paths,
        batch_size=batch_size,
        patch_size=(256, 256),
        n_channels=3,
        n_classes=num_classes,
        rs=rs,
        rs_label=rs_label,
        shuffle=True
    )
    
    valid_gen = DataGenerator(
        valid_image_paths, 
        valid_label_paths,
        batch_size=batch_size,
        patch_size=(256, 256),
        n_channels=3,
        n_classes=num_classes,
        rs=rs,
        rs_label=rs_label,
        shuffle=False
    )
    
    train_steps = len(train_image_paths) // batch_size
    valid_steps = len(valid_image_paths) // batch_size
    
    # Training the model
    model.fit(
        train_gen,
        steps_per_epoch=train_steps,
        validation_data=valid_gen,
        validation_steps=valid_steps,
        epochs=epochs
    )
    
    print("Model successfully trained")

    return model

def evaluate_model(model, test_image_paths, test_label_paths, batch_size, rs, rs_label):
    test_gen = DataGenerator(
        test_image_paths, 
        test_label_paths,
        batch_size=batch_size,
        patch_size=(256, 256),
        n_channels=3,
        n_classes=1,
        rs=rs,
        rs_label=rs_label,
        shuffle=False
    )

    # Get predictions from the model
    predictions = model.predict(test_gen, verbose=1)

    # Flatten predictions and true labels
    pred_masks = (predictions > 0.5).astype(np.uint8).flatten()
    true_masks = np.concatenate([test_gen[i][1].flatten() for i in range(len(test_gen))]).astype(np.uint8)

    pred_masks = np.clip(pred_masks, 0, 1)
    true_masks = np.clip(true_masks, 0, 1)

    # Calculate accuracy
    accuracy = accuracy_score(true_masks, pred_masks)

    # Calculate F1-score
    f1 = f1_score(true_masks, pred_masks, average='binary')

    # Calculate mean intersection over union (mIoU)
    intersection = np.logical_and(true_masks, pred_masks)
    union = np.logical_or(true_masks, pred_masks)
    iou = np.sum(intersection) / np.sum(union)

    print("Evaluation Metrics:")
    print(f"Accuracy: {accuracy}")
    print(f"F1-score: {f1}")
    print(f"Mean Intersection over Union (mIoU): {iou}")

# Parameters
model_name = "unet"
input_shape = (256, 256, 3)
batch_size = 16
num_classes = 1  # Binary segmentation
epochs = 10
rs = 8
rs_label = 1
weights = None

# Train the model
model = train(model_name, train_image_paths, train_label_paths, valid_image_paths, valid_label_paths, 
              input_shape, batch_size, num_classes, epochs, rs, rs_label, weights)

# Evaluate the model
evaluate_model(model, test_image_paths, test_label_paths, batch_size, rs, rs_label)


Epoch 1/10
2/2 [==============================] - 9s 5s/step - loss: 1469.9866 - accuracy: 0.0734
Epoch 2/10
2/2 [==============================] - 9s 5s/step - loss: -1481.5182 - accuracy: 0.0595
Epoch 3/10
2/2 [==============================] - 9s 5s/step - loss: -5454.3447 - accuracy: 0.0643
Epoch 4/10
2/2 [==============================] - 10s 5s/step - loss: -7585.3638 - accuracy: 0.0649
Epoch 5/10
2/2 [==============================] - 9s 5s/step - loss: -5162.6494 - accuracy: 0.1117
Epoch 6/10
2/2 [==============================] - 9s 5s/step - loss: -11164.2441 - accuracy: 0.0520
Epoch 7/10
2/2 [==============================] - 10s 5s/step - loss: -12915.0615 - accuracy: 0.0541
Epoch 8/10
2/2 [==============================] - 9s 5s/step - loss: -10725.3145 - accuracy: 0.1032
Epoch 9/10
2/2 [==============================] - 9s 5s/step - loss: -15015.9492 - accuracy: 0.0508
Epoch 10/10
2/2 [==============================] - 9s 5s/step - loss: -16081.4902 - accuracy: 0.0423
Mod